# Mispriced Bonds

#### This program finds mispriced bonds using input spot curve.
#### Discount bonds using spot curve and compare them with market prices

1. Part 1 use linear interpolated spot curve using Spot_Curve_Linear_Interpolation.ipynb. Inital data from bonds.
2. Part 2 use linear interpolated spot curve. Initial data from swaps.
3. Part 3 use Cubic interpolated spot curve. Initial data from bonds.

In [1]:
import pandas as pd
import numpy as np

#### 1. Linear Interpolated Spot Curve

##### 1.1 Import spot curve data

In [2]:
spotcurve=pd.read_excel("C:/Users/yuxiang/Desktop/fiexed_income_project/spotcurve.xlsx")
spotcurve

,Maturity,Coupon,Ask Price,year,month,n,spot
1,8/15/2019,2.166667,99.8112,2019,8,1,2.166667
2,2/29/2020,2.250000,99.6914,2020,2,2,2.250000
3,8/15/2020,2.458333,99.8672,2020,8,3,2.458333
4,2/15/2021,2.250000,99.4609,2021,2,4,2.250000
5,8/15/2021,2.750000,100.578,2021,8,5,2.750000
6,2/15/2022,2.500000,100,2022,2,6,2.500000
7,/,2.562500,/,2022,8,7,2.566385
8,2/28/2023,2.625000,100.445,2023,2,8,2.625000
9,8/15/2023,2.500000,99.9531,2023,8,9,2.500000
10,/,2.527778,/,2024,2,10,2.528556


##### 1.2 Import bonds data

In [3]:
bond=pd.read_excel("C:/Users/yuxiang/Desktop/fiexed_income_project/bond.xlsx")

bond['Maturity']=pd.to_datetime(bond['Maturity'], format='%m/%d/%Y')
bond=bond.sort_values('Maturity')
bond=bond.reset_index().iloc[:,1:]
bond['time']=float(0)
for i in range(len(bond)):  #i=311
    bond['time'][i]=float((bond.Maturity[i]-bond.Maturity[0]).days)/365
bond=bond.loc[bond['time']>0.5]
bond=bond.loc[bond['time']<29.5]
bond=bond.reset_index().iloc[:,1:]
bond

d:\program files\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Maturity,Coupon,Ask Price,Identifier,time
0,2019-08-31,1.625,99.519531,912828D80,0.539726
1,2019-08-31,1.250,99.308594,9128282T6,0.539726
2,2019-08-31,1.000,99.164062,912828TN0,0.539726
3,2019-09-15,0.875,99.027344,9128282G4,0.580822
4,2019-09-30,1.750,99.511719,912828F39,0.621918
5,2019-09-30,1.375,99.281250,9128282X7,0.621918
6,2019-09-30,1.000,99.031250,912828TR1,0.621918
7,2019-10-15,1.000,98.972656,912828T59,0.663014
8,2019-10-31,1.250,99.082031,912828TV2,0.706849
9,2019-10-31,1.500,99.261719,912828F62,0.706849


##### 1.3 Discount bonds using spot curve

Define a function return spot rate

In [4]:
def spot_func(x):
    if 0<x<1:
        return 2.1;
    return spotcurve.spot[int(x)]+(x%1)*(spotcurve.spot[int(x)+1]-spotcurve.spot[int(x)])

Discount, estimate price

In [5]:
bond['price_est']=0.0
for i in range(len(bond)):  # t=bond.loc[1]  i=2
    coupon_t=[(bond.time[i]*2)%1 + i for i in range(int(bond.time[i]*2)+1)]    
    bond['price_est'][i]=sum([bond.Coupon[i]/2*(1/(1+spot_func(j)/200))**j for j in coupon_t]+[100*(1/(1+spot_func(bond.time[i]*2)/200))**(bond.time[i]*2)])
bond

d:\program files\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Maturity,Coupon,Ask Price,Identifier,time,price_est
0,2019-08-31,1.625,99.519531,912828D80,0.539726,100.455028
1,2019-08-31,1.250,99.308594,9128282T6,0.539726,100.082359
2,2019-08-31,1.000,99.164062,912828TN0,0.539726,99.833912
3,2019-09-15,0.875,99.027344,9128282G4,0.580822,99.618041
4,2019-09-30,1.750,99.511719,912828F39,0.621918,100.395491
5,2019-09-30,1.375,99.281250,9128282X7,0.621918,100.022821
6,2019-09-30,1.000,99.031250,912828TR1,0.621918,99.650151
7,2019-10-15,1.000,98.972656,912828T59,0.663014,99.557576
8,2019-10-31,1.250,99.082031,912828TV2,0.706849,99.706767
9,2019-10-31,1.500,99.261719,912828F62,0.706849,99.955213


##### 1.4 Misprice

In [6]:
bond['Market_minus_estimate']=bond['Ask Price']-bond['price_est']
bond['misprice_percent']=bond.Market_minus_estimate/bond['Ask Price']*100
bond['misprice']=bond['misprice_percent']/abs(bond['misprice_percent'])
print('standard deviation of misprice percentile from market price:',np.std(bond['misprice_percent']))

standard deviation of misprice percentile from market price: 0.6969040750172428


In [7]:
bond

,Maturity,Coupon,Ask Price,Identifier,time,price_est,Market_minus_estimate,misprice_percent,misprice
0,2019-08-31,1.625,99.519531,912828D80,0.539726,100.455028,-0.935497,-0.940014,-1.0
1,2019-08-31,1.250,99.308594,9128282T6,0.539726,100.082359,-0.773765,-0.779152,-1.0
2,2019-08-31,1.000,99.164062,912828TN0,0.539726,99.833912,-0.669850,-0.675496,-1.0
3,2019-09-15,0.875,99.027344,9128282G4,0.580822,99.618041,-0.590697,-0.596499,-1.0
4,2019-09-30,1.750,99.511719,912828F39,0.621918,100.395491,-0.883772,-0.888108,-1.0
5,2019-09-30,1.375,99.281250,9128282X7,0.621918,100.022821,-0.741571,-0.746940,-1.0
6,2019-09-30,1.000,99.031250,912828TR1,0.621918,99.650151,-0.618901,-0.624956,-1.0
7,2019-10-15,1.000,98.972656,912828T59,0.663014,99.557576,-0.584920,-0.590991,-1.0
8,2019-10-31,1.250,99.082031,912828TV2,0.706849,99.706767,-0.624735,-0.630523,-1.0
9,2019-10-31,1.500,99.261719,912828F62,0.706849,99.955213,-0.693494,-0.698652,-1.0


#### 2. Linear Interpolated Spot Curve Using Swaps

##### 2.1 Import spot curve data

In [8]:
swap=pd.read_excel("C:/Users/yuxiang/Desktop/fiexed_income_project/swap.xlsx")
swap.index+=1  # take care of this
swap

,n,spot
1,1,2.7260
2,2,2.7260
3,3,2.6965
4,4,2.6670
5,5,2.6410
6,6,2.6150
7,7,2.6050
8,8,2.5950
9,9,2.5980
10,10,2.6010


##### 2.2 Import bonds data

In [9]:
bond_swap=pd.read_excel("C:/Users/yuxiang/Desktop/fiexed_income_project/bond.xlsx")

bond_swap['Maturity']=pd.to_datetime(bond_swap['Maturity'], format='%m/%d/%Y')
bond_swap=bond_swap.sort_values('Maturity')
bond_swap=bond_swap.reset_index().iloc[:,1:]
bond_swap['time']=float(0)
for i in range(len(bond_swap)):  #i=311
    bond_swap['time'][i]=float((bond_swap.Maturity[i]-bond_swap.Maturity[0]).days)/365
bond_swap=bond_swap.loc[bond_swap['time']>0.5]
bond_swap=bond_swap.loc[bond_swap['time']<29.5]
bond_swap=bond_swap.reset_index().iloc[:,1:]
bond_swap

d:\program files\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Maturity,Coupon,Ask Price,Identifier,time
0,2019-08-31,1.625,99.519531,912828D80,0.539726
1,2019-08-31,1.250,99.308594,9128282T6,0.539726
2,2019-08-31,1.000,99.164062,912828TN0,0.539726
3,2019-09-15,0.875,99.027344,9128282G4,0.580822
4,2019-09-30,1.750,99.511719,912828F39,0.621918
5,2019-09-30,1.375,99.281250,9128282X7,0.621918
6,2019-09-30,1.000,99.031250,912828TR1,0.621918
7,2019-10-15,1.000,98.972656,912828T59,0.663014
8,2019-10-31,1.250,99.082031,912828TV2,0.706849
9,2019-10-31,1.500,99.261719,912828F62,0.706849


##### 2.3 Discount bonds using spot curve

Define a function return spot rate

In [10]:
def spot_func_swap(x):
    if 0<x<1:
        return 2.7260;
    return swap.spot[int(x)]+(x%1)*(swap.spot[int(x)+1]-swap.spot[int(x)])

Discount, estimate price

In [11]:
bond_swap['price_est']=0.0
for i in range(len(bond_swap)):  # t=bond_swap.loc[1]  i=2
    coupon_t=[(bond_swap.time[i]*2)%1 + i for i in range(int(bond_swap.time[i]*2)+1)]    
    bond_swap['price_est'][i]=sum([bond_swap.Coupon[i]/2*(1/(1+spot_func_swap(j)/200))**j for j in coupon_t]+[100*(1/(1+spot_func_swap(bond_swap.time[i]*2)/200))**(bond_swap.time[i]*2)])
bond_swap

d:\program files\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Maturity,Coupon,Ask Price,Identifier,time,price_est
0,2019-08-31,1.625,99.519531,912828D80,0.539726,100.161609
1,2019-08-31,1.250,99.308594,9128282T6,0.539726,99.789531
2,2019-08-31,1.000,99.164062,912828TN0,0.539726,99.541478
3,2019-09-15,0.875,99.027344,9128282G4,0.580822,99.307857
4,2019-09-30,1.750,99.511719,912828F39,0.621918,100.066566
5,2019-09-30,1.375,99.281250,9128282X7,0.621918,99.694487
6,2019-09-30,1.000,99.031250,912828TR1,0.621918,99.322409
7,2019-10-15,1.000,98.972656,912828T59,0.663014,99.213057
8,2019-10-31,1.250,99.082031,912828TV2,0.706849,99.344601
9,2019-10-31,1.500,99.261719,912828F62,0.706849,99.592653


##### 2.4 Misprice

In [12]:
bond_swap['Market_minus_estimate']=bond_swap['Ask Price']-bond_swap['price_est']
bond_swap['misprice_percent']=bond_swap.Market_minus_estimate/bond_swap['Ask Price']*100
bond_swap['misprice']=bond_swap['misprice_percent']/abs(bond_swap['misprice_percent'])
print('standard deviation of misprice percentile from market price:',np.std(bond_swap['misprice_percent']))

standard deviation of misprice percentile from market price: 1.385267647228479


In [13]:
bond_swap

,Maturity,Coupon,Ask Price,Identifier,time,price_est,Market_minus_estimate,misprice_percent,misprice
0,2019-08-31,1.625,99.519531,912828D80,0.539726,100.161609,-0.642078,-0.645177,-1.0
1,2019-08-31,1.250,99.308594,9128282T6,0.539726,99.789531,-0.480937,-0.484285,-1.0
2,2019-08-31,1.000,99.164062,912828TN0,0.539726,99.541478,-0.377416,-0.380597,-1.0
3,2019-09-15,0.875,99.027344,9128282G4,0.580822,99.307857,-0.280513,-0.283268,-1.0
4,2019-09-30,1.750,99.511719,912828F39,0.621918,100.066566,-0.554847,-0.557569,-1.0
5,2019-09-30,1.375,99.281250,9128282X7,0.621918,99.694487,-0.413237,-0.416229,-1.0
6,2019-09-30,1.000,99.031250,912828TR1,0.621918,99.322409,-0.291159,-0.294007,-1.0
7,2019-10-15,1.000,98.972656,912828T59,0.663014,99.213057,-0.240401,-0.242896,-1.0
8,2019-10-31,1.250,99.082031,912828TV2,0.706849,99.344601,-0.262570,-0.265003,-1.0
9,2019-10-31,1.500,99.261719,912828F62,0.706849,99.592653,-0.330935,-0.333396,-1.0


#### 3. Cubic Interpolated Spot Curve

##### 3.1 Import spot curve data

In [14]:
cubic=pd.read_excel("C:/Users/yuxiang/Desktop/fiexed_income_project/cubic.xlsx")
cubic.index+=1  # take care of this
cubic

,n,spot
1,1,2.166667
2,2,2.250000
3,3,2.458333
4,4,2.250000
5,5,2.750000
6,6,2.500000
7,7,2.218750
8,8,2.625000
9,9,2.500000
10,10,2.301852


##### 3.2 Import bonds data

In [15]:
bond_cubic=pd.read_excel("C:/Users/yuxiang/Desktop/fiexed_income_project/bond.xlsx")

bond_cubic['Maturity']=pd.to_datetime(bond_cubic['Maturity'], format='%m/%d/%Y')
bond_cubic=bond_cubic.sort_values('Maturity')
bond_cubic=bond_cubic.reset_index().iloc[:,1:]
bond_cubic['time']=float(0)
for i in range(len(bond_cubic)):  #i=311
    bond_cubic['time'][i]=float((bond_cubic.Maturity[i]-bond_cubic.Maturity[0]).days)/365
bond_cubic=bond_cubic.loc[bond_cubic['time']>0.5]
bond_cubic=bond_cubic.loc[bond_cubic['time']<28]
bond_cubic=bond_cubic.reset_index().iloc[:,1:]
bond_cubic

d:\program files\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Maturity,Coupon,Ask Price,Identifier,time
0,2019-08-31,1.625,99.519531,912828D80,0.539726
1,2019-08-31,1.250,99.308594,9128282T6,0.539726
2,2019-08-31,1.000,99.164062,912828TN0,0.539726
3,2019-09-15,0.875,99.027344,9128282G4,0.580822
4,2019-09-30,1.750,99.511719,912828F39,0.621918
5,2019-09-30,1.375,99.281250,9128282X7,0.621918
6,2019-09-30,1.000,99.031250,912828TR1,0.621918
7,2019-10-15,1.000,98.972656,912828T59,0.663014
8,2019-10-31,1.250,99.082031,912828TV2,0.706849
9,2019-10-31,1.500,99.261719,912828F62,0.706849


##### 3.3 Discount bonds using spot curve

Define a function return spot rate

In [16]:
def spot_func_cubic(x):
    if 0<x<1:
        return 2.1;
    return cubic.spot[int(x)]+(x%1)*(cubic.spot[int(x)+1]-cubic.spot[int(x)])

Discount, estimate price

In [17]:
bond_cubic['price_est']=0.0
for i in range(len(bond_cubic)):  # t=bond_cubic.loc[1]  i=2
    coupon_t=[(bond_cubic.time[i]*2)%1 + i for i in range(int(bond_cubic.time[i]*2)+1)]    
    bond_cubic['price_est'][i]=sum([bond_cubic.Coupon[i]/2*(1/(1+spot_func_cubic(j)/200))**j for j in coupon_t]+[100*(1/(1+spot_func_cubic(bond_cubic.time[i]*2)/200))**(bond_cubic.time[i]*2)])
bond_cubic

d:\program files\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Maturity,Coupon,Ask Price,Identifier,time,price_est
0,2019-08-31,1.625,99.519531,912828D80,0.539726,100.455028
1,2019-08-31,1.250,99.308594,9128282T6,0.539726,100.082359
2,2019-08-31,1.000,99.164062,912828TN0,0.539726,99.833912
3,2019-09-15,0.875,99.027344,9128282G4,0.580822,99.618041
4,2019-09-30,1.750,99.511719,912828F39,0.621918,100.395491
5,2019-09-30,1.375,99.281250,9128282X7,0.621918,100.022821
6,2019-09-30,1.000,99.031250,912828TR1,0.621918,99.650151
7,2019-10-15,1.000,98.972656,912828T59,0.663014,99.557576
8,2019-10-31,1.250,99.082031,912828TV2,0.706849,99.706767
9,2019-10-31,1.500,99.261719,912828F62,0.706849,99.955213


##### 3.4 Misprice

In [18]:
bond_cubic['Market_minus_estimate']=bond_cubic['Ask Price']-bond_cubic['price_est']
bond_cubic['misprice_percent']=bond_cubic.Market_minus_estimate/bond_cubic['Ask Price']*100
bond_cubic['misprice']=bond_cubic['misprice_percent']/abs(bond_cubic['misprice_percent'])
print('standard deviation of misprice percentile from market price:',np.std(bond_cubic['misprice_percent']))

standard deviation of misprice percentile from market price: 10.0515610264404


In [19]:
bond_cubic

,Maturity,Coupon,Ask Price,Identifier,time,price_est,Market_minus_estimate,misprice_percent,misprice
0,2019-08-31,1.625,99.519531,912828D80,0.539726,100.455028,-0.935497,-0.940014,-1.0
1,2019-08-31,1.250,99.308594,9128282T6,0.539726,100.082359,-0.773765,-0.779152,-1.0
2,2019-08-31,1.000,99.164062,912828TN0,0.539726,99.833912,-0.669850,-0.675496,-1.0
3,2019-09-15,0.875,99.027344,9128282G4,0.580822,99.618041,-0.590697,-0.596499,-1.0
4,2019-09-30,1.750,99.511719,912828F39,0.621918,100.395491,-0.883772,-0.888108,-1.0
5,2019-09-30,1.375,99.281250,9128282X7,0.621918,100.022821,-0.741571,-0.746940,-1.0
6,2019-09-30,1.000,99.031250,912828TR1,0.621918,99.650151,-0.618901,-0.624956,-1.0
7,2019-10-15,1.000,98.972656,912828T59,0.663014,99.557576,-0.584920,-0.590991,-1.0
8,2019-10-31,1.250,99.082031,912828TV2,0.706849,99.706767,-0.624735,-0.630523,-1.0
9,2019-10-31,1.500,99.261719,912828F62,0.706849,99.955213,-0.693494,-0.698652,-1.0
